In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

## 7.2.1. The Cross-Correlation Operation

Recall that strictly speaking, convolutional layers are a misnomer, since the operations they express are more accurately described as **cross-correlations**. Based on our descriptions of convolutional layers in Section 7.1, in such a layer, an input tensor and a kernel tensor are combined to produce an output tensor through a cross-correlation operation.

实际上，CNN中用到的卷积计算，严格来说都是互相关运算。

In [ ]:
def corr2d(x, kernel):  #@save
    """Compute 2D cross-correlation."""
    # h: height
    # w: width
    x_h, x_w = x.shape
    k_h, k_w = kernel.shape
    y = torch.zeros((x_h - k_h + 1, x_w - k_w + 1))
    y_h, y_w = y.shape
    for i in range(y_h):
        for j in range(y_w):
            y[i, j] = (x[i:i + k_h, j:j + k_w] * kernel).sum()
    return y

In [ ]:
x = torch.tensor([[0.0, 1.0, 2.0],
                  [3.0, 4.0, 5.0],
                  [6.0, 7.0, 8.0]])
x

In [ ]:
kernel = torch.tensor([[0.0, 1.0],
                       [2.0, 3.0]])
kernel

In [ ]:
corr2d(x, kernel)

## 7.2.2. Convolutional Layers

In [ ]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 7.2.3. Object Edge Detection in Images

In [ ]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

In [ ]:
# construct a kernel to detect edge in this image
kernel = torch.tensor([[1.0, -1.0]])
# after the convolution calculation, the same value takes the value 0

In [ ]:
Y = corr2d(X, kernel)

In [ ]:
# this kernel can only detect vertical edge
corr2d(X.t(), kernel)

## 7.2.4. Learning a Kernel

In [ ]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

In [ ]:
# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
X

In [ ]:
Y = Y.reshape((1, 1, 6, 7))
Y

In [ ]:
lr = 3e-2  # Learning rate
loss = 0
for i in range(10):
    Y_hat = conv2d(X)
    # MSE作为loss function
    loss = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    loss.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {loss.sum():.3f}')

In [ ]:
conv2d.weight.data.reshape((1, 2))

## 7.2.8. Exercises

Design some kernels manually.

1. detects edges orthogonal（正交的，垂直的）